- exp005&exp006

In [13]:
import pandas as pd
from pathlib import Path

class CFG:
    name = "ens001"
    seed = 42
    n_fold = 5

    path_input = Path("../input")
    path_output = Path("../output")

In [14]:
def setup(CFG):
    # expフォルダを作成
    CFG.path_exp = CFG.path_output / CFG.name
    CFG.path_exp.mkdir(parents=True, exist_ok=True)

    return CFG

CFG = setup(CFG)

In [15]:
sub_exp005 = pd.read_csv("../output/exp005/submission.csv")
sub_exp006 = pd.read_csv("../output/exp006/submission.csv")

In [16]:
sub_exp005['predict'] = sub_exp005.apply(lambda row: ' '.join(row.astype(str)), axis=1)
sub_exp006['predict'] = sub_exp006.apply(lambda row: ' '.join(row.astype(str)), axis=1)

In [17]:
def cust_blend(dt, W = [1,1,1]):
    REC = []
    REC.append(dt['predict_0'].split())
    REC.append(dt['predict_1'].split())

    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 10 itens only
    return ' '.join(res[:10])

In [18]:
sub = sub_exp005[['predict']]
sub['predict1'] = sub_exp006['predict']
sub.columns = [f'predict_{i}' for i in range(len(sub.columns))]

/var/folders/3l/2bkf0scn43g79fkq6f3ft9z40000gn/T/ipykernel_62163/3224590921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['predict1'] = sub_exp006['predict']


In [19]:
sub

,predict_0,predict_1
0,3560 4420 4714 10965 2680 9830 5466 11561 1193...,3560 4714 2680 4420 11561 5466 9830 2040 2305 ...
1,6129 4066 6555 143 12862 613 7913 7014 11237 1...,143 4066 6555 7014 7913 6129 8108 11237 12862 613
2,757 6721 410 7710 10485 9190 9910 1774 3400 6730,757 757 7710 9190 10485 410 1774 3400 6721 6730
3,12341 3359 6991 10746 5080 277 5657 13022 2795...,12341 3359 12341 5080 6991 10746 13521 277 154...
4,3854 9020 12029 3844 5411 2862 5372 3476 6161 ...,9020 2862 3476 3844 3854 4070 5372 5411 6161 6565
...,...,...
174695,1997 3440 9743 7888 2278 5744 7780 10997 1885 ...,1997 1997 2278 5744 7062 7888 9543 9743 10997 ...
174696,1227 12432 4014 2164 899 13702 6874 3802 13220...,12432 1227 3802 899 2164 3644 4014 13220 13702...
174697,8143 5810 13241 11796 844 2087 12425 12939 737...,13241 844 2087 5810 7308 7379 8143 11796 12240...
174698,11496 2692 2373 10287 3100 2305 5079 2510 5513...,2692 3100 10287 2305 2373 2510 3002 3096 4976 ...


In [20]:
# 今回はweightを1:1とする。
sub['predict'] = sub.apply(cust_blend, W = [2.00,1.00], axis=1)

/var/folders/3l/2bkf0scn43g79fkq6f3ft9z40000gn/T/ipykernel_62163/1513419769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['predict'] = sub.apply(cust_blend, W = [2.00,1.00], axis=1)


In [21]:
ensemble_sub = sub['predict'].str.split(' ',expand=True)

In [22]:
ensemble_sub.columns = [f'predict_{i}' for i in range(ensemble_sub.shape[1])]


In [23]:
ensemble_sub

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,3560,4420,4714,2680,10965,9830,5466,11561,11930,2986
1,6129,4066,143,6555,12862,7014,7913,613,11237,11923
2,757,6721,410,7710,10485,9190,1774,3400,6730,9910
3,12341,3359,6991,10746,5080,277,5657,13022,2795,6489
4,3854,9020,2862,3844,12029,3476,5411,5372,6161,9623
...,...,...,...,...,...,...,...,...,...,...
174695,1997,3440,9743,2278,7888,5744,10997,7780,1885,11848
174696,1227,12432,4014,2164,899,3802,13702,13220,6874,5331
174697,8143,13241,5810,844,2087,11796,7379,12939,12240,12425
174698,11496,2692,3100,2373,10287,2305,2510,5079,3002,5513


In [24]:
ensemble_sub.to_csv(CFG.path_exp / "submission.csv", index=False)